In [ ]:
import pandas as pd

from utils.conditioning import get_conditioning_deltas
from utils.routing import get_route_factor_adjustments, get_route_number
from utils.packaging import get_packaging_changes, construct_package_routes
from utils.material_mass import calculate_material_mass,calculate_haz_material_mass
from utils.activity_limits import evaluate_activity_limits

In [ ]:
filename = "packages_input.xlsx"
data = pd.read_excel(filename, skiprows=[0], sheet_name="Data")
routing_factors = pd.read_excel(filename, sheet_name="Routing").fillna(0)
packages = pd.read_excel(filename, sheet_name="Packages", skiprows=[0])
rads = pd.read_excel(filename, sheet_name="Rads", skiprows=[0])

radionucleide_column_indices = range(9, 12) # 0-indexed column numbers for any radionucleide activity values
radionucleide_cols = [data.columns[c] for c in radionucleide_column_indices]

# The packages sheet can be simplified by taking only the primary package definitions and defining secondary state packaging seperately
packages = packages[list(packages.columns[:8]) + [packages.columns[-1]]]
package_routes = [("Drum", "Stillage"),
                  ("Shielded drum", "No Package"),
                  ("4m box", "No Package"),
                  ("3m box", "No Package"),
                  ("No Package", "No Package"),
                  ("Stillage", "No Package")]

package_routes_df = construct_package_routes(package_routes, packages)

In [ ]:
data["Route Number"] = get_route_number(data["Waste Stream"])
data["Mass"] = data["Density"] * data["Volume"]
data.fillna({c: 0 for c in radionucleide_cols}, inplace=True)

unpackaged_type = "No Package"
data["Package type"].fillna(unpackaged_type, inplace=True)

# Any remaining null conditioning factors must be set according to package type
default_factors = data.merge(packages, left_on="Package type", right_on="Package type", how="left")[["Default conditioning factor", "Default packing factor"]]
data["Conditioning factor"].fillna(default_factors["Default conditioning factor"], inplace=True)

# Any null packaging factors must be set according to package type
data["Packaging factor"].fillna(default_factors["Default packing factor"], inplace=True)

In [ ]:
hazardous_material_masses = calculate_haz_material_mass(data, hazardous_material_col=['Haz'])

non_hazardous_material_cols = ['Steel', 'Not steel']
non_hazardous_material_masses = calculate_material_mass(streams=data, non_hazardous_material_cols=non_hazardous_material_cols)

In [ ]:
animal_radioactivity_limits = rads.set_index('Rad')['Animals']
animal_radioactivity_limits = pd.to_numeric(animal_radioactivity_limits,errors= 'coerce')

In [ ]:
animal_activity_limits =  evaluate_activity_limits(data, radionucleide_cols, animal_radioactivity_limits)
animal_activity_limits['Waste Stream'] = data['Waste Stream']

In [ ]:
route_adjusted_data = data.copy(deep=True) # make a copy so we can preserve output at each step

route_adjustment_values = get_route_factor_adjustments(streams=data, route_factors=routing_factors, radionucleide_cols=radionucleide_cols)
route_adjusted_data[route_adjustment_values.columns] = route_adjustment_values
route_adjusted_data

route_adjusted_data[non_hazardous_material_cols] = non_hazardous_material_masses[non_hazardous_material_cols]
route_adjusted_data['Haz'] = hazardous_material_masses['Haz']

In [ ]:
conditional_changes = get_conditioning_deltas(route_adjusted_data)

conditioned_data = route_adjusted_data.copy(deep=True)
conditioned_data["Volume"] += conditional_changes["Volume Delta"]
conditioned_data["Mass"] += conditional_changes["Mass Delta"]
conditioned_data["Conditioning material"] = conditional_changes["Conditioning material"]

In [ ]:
packing_changes = get_packaging_changes(conditioned_data, package_routes_df)

packed_data = conditioned_data.copy(deep=True)
packed_data["Volume"] = packing_changes["Secondary Packaged Volume"]
packed_data["Mass"] = packed_data["Mass"] + packing_changes["Primary Mass Delta"] + packing_changes["Secondary Mass Delta"]

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:
    data.to_excel(writer, sheet_name='input')
    conditioned_data.to_excel(writer, sheet_name='conditioned')
    packed_data.to_excel(writer, sheet_name='packaged')
    animal_activity_limits.to_excel(writer, sheet_name= 'SoF assessment')